In [1]:
!pip install --quiet optuna

     |████████████████████████████████| 348 kB 8.1 MB/s 
     |████████████████████████████████| 81 kB 10.0 MB/s 
     |████████████████████████████████| 209 kB 66.8 MB/s 
     |████████████████████████████████| 78 kB 7.1 MB/s 
     |████████████████████████████████| 49 kB 6.7 MB/s 
     |████████████████████████████████| 112 kB 71.9 MB/s 
     |████████████████████████████████| 147 kB 76.1 MB/s 


In [2]:
import numpy as np
import pandas as pd
import optuna as opt
import xgboost as xgb
from optuna.samplers import TPESampler
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_score
from pathlib import Path
import gc

from sklearn.metrics import mean_absolute_error, mean_squared_error

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
RANDOM_SEED = 2
DATA_DIR = Path("/content/drive/MyDrive/CS760")

In [5]:
df_train = pd.read_parquet(DATA_DIR/"train_main.parquet.snappy")
df_test = pd.read_parquet(DATA_DIR/"test_main.parquet.snappy")

print(f"Shape of the training data : {df_train.shape}")
print(f"Shape of the test data : {df_test.shape}")

Shape of the training data : (400000, 13)
Shape of the test data : (100000, 13)


In [6]:
df_train.describe().T

,count,mean,std,min,25%,50%,75%,max
r_id,400000.0,3.525889e+06,2.018260e+06,1.900000e+01,1.807686e+06,3.486237e+06,5.294850e+06,6.990280e+06
r_stars,400000.0,3.607683e+00,1.539312e+00,1.000000e+00,2.000000e+00,4.000000e+00,5.000000e+00,5.000000e+00
r_stars_square,400000.0,1.538485e+01,9.625009e+00,1.000000e+00,4.000000e+00,1.600000e+01,2.500000e+01,2.500000e+01
r_length,400000.0,1.332325e+02,1.149695e+02,0.000000e+00,5.700000e+01,1.010000e+02,1.700000e+02,1.068000e+03
u_friends_count,400000.0,1.899421e+02,5.735422e+02,1.000000e+00,2.000000e+00,4.000000e+01,1.680000e+02,1.499500e+04
u_review_count,400000.0,1.915879e+02,5.024456e+02,0.000000e+00,1.100000e+01,4.400000e+01,1.760000e+02,1.747300e+04
u_month_age,400000.0,4.277618e+01,3.456388e+01,3.802649e-07,1.362863e+01,3.651231e+01,6.514719e+01,2.017248e+02
b_stars,400000.0,3.744636e+00,7.930024e-01,1.000000e+00,3.500000e+00,4.000000e+00,4.500000e+00,5.000000e+00
b_review_count,400000.0,3.346235e+02,6.589277e+02,5.000000e+00,3.900000e+01,1.250000e+02,3.410000e+02,7.568000e+03
r_sen,400000.0,1.821748e-01,1.927162e-01,-1.000000e+00,6.000000e-02,1.805560e-01,3.001218e-01,1.000000e+00


In [7]:
X_train, y_train = df_train.drop(['r_useful', 'r_id'], axis=1).values, df_train['r_useful'].values
X_test, y_test = df_test.drop(['r_useful', 'r_id'], axis=1).values, df_test['r_useful'].values

In [8]:
def objective(trial):
  """ Function to tune parameters """
  gc.collect()
  params = {
      "n_estimators":trial.suggest_int('n_estimators', 1, 1001, step=50),
      "max_depth" : trial.suggest_int("max_depth", 2, 20),
      "learning_rate" : trial.suggest_float('learning_rate', 0.001, 0.5, log=True),
      "subsample ": trial.suggest_float("subsample", 0.4, 1, step=0.1),
      "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1, step=0.1),
      "gamma": trial.suggest_float("gamma", 0, 1),
      "min_child_weight": trial.suggest_int("min_child_weight", 1, 31, step=2)
  }

  print("Currently running with:")
  print(params)

  model = xgb.XGBRegressor(objective="reg:squarederror",
                            n_jobs=-1,
                            grow_policy='lossguide',
                            tree_method="gpu_hist",
                            predictor="gpu_predictor",
                            booster='gbtree',
                            sampling_method='gradient_based',
                            eval_metrics=['rmse'],
                            random_state=RANDOM_SEED,
                            enable_categorical=False,
                            **params)
  pipe = make_pipeline(StandardScaler(), model)

  scores = cross_val_score(pipe, X_train, y_train,
    scoring="neg_root_mean_squared_error", cv=KFold(3))

  return -scores.mean()

study = opt.create_study(direction='minimize', sampler=TPESampler(seed=RANDOM_SEED))
study.optimize(objective, n_trials=50)
print("Final best parameters:")
study.best_params

[I 2022-10-01 00:20:06,849] A new study created in memory with name: no-name-e6887848-4f54-4901-88c8-8badddd76115


Currently running with:
{'n_estimators': 451, 'max_depth': 2, 'learning_rate': 0.030445460008040798, 'subsample ': 0.7000000000000001, 'colsample_bytree': 0.6000000000000001, 'gamma': 0.3303348210038741, 'min_child_weight': 7}


[I 2022-10-01 00:20:17,491] Trial 0 finished with value: 3.7135762047750642 and parameters: {'n_estimators': 451, 'max_depth': 2, 'learning_rate': 0.030445460008040798, 'subsample': 0.7000000000000001, 'colsample_bytree': 0.6000000000000001, 'gamma': 0.3303348210038741, 'min_child_weight': 7}. Best is trial 0 with value: 3.7135762047750642.


Currently running with:
{'n_estimators': 651, 'max_depth': 7, 'learning_rate': 0.005249994057689375, 'subsample ': 0.8, 'colsample_bytree': 0.7000000000000001, 'gamma': 0.13457994534493356, 'min_child_weight': 17}


[I 2022-10-01 00:20:40,447] Trial 1 finished with value: 3.5931423895160304 and parameters: {'n_estimators': 651, 'max_depth': 7, 'learning_rate': 0.005249994057689375, 'subsample': 0.8, 'colsample_bytree': 0.7000000000000001, 'gamma': 0.13457994534493356, 'min_child_weight': 17}. Best is trial 1 with value: 3.5931423895160304.


...

[I 2022-10-01 01:44:20,728] Trial 48 finished with value: 3.5160827346546153 and parameters: {'n_estimators': 851, 'max_depth': 10, 'learning_rate': 0.005580914718472491, 'subsample': 1.0, 'colsample_bytree': 0.7000000000000001, 'gamma': 0.12364625880766489, 'min_child_weight': 3}. Best is trial 47 with value: 3.4898353435661043.


Currently running with:
{'n_estimators': 751, 'max_depth': 7, 'learning_rate': 0.36190245986996133, 'subsample ': 1.0, 'colsample_bytree': 0.7000000000000001, 'gamma': 0.17899549479454283, 'min_child_weight': 1}


[I 2022-10-01 01:44:45,865] Trial 49 finished with value: 3.714591373764179 and parameters: {'n_estimators': 751, 'max_depth': 7, 'learning_rate': 0.36190245986996133, 'subsample': 1.0, 'colsample_bytree': 0.7000000000000001, 'gamma': 0.17899549479454283, 'min_child_weight': 1}. Best is trial 47 with value: 3.4898353435661043.


Final best parameters:


{'n_estimators': 851,
 'max_depth': 10,
 'learning_rate': 0.009707298411215315,
 'subsample': 1.0,
 'colsample_bytree': 0.7000000000000001,
 'gamma': 0.1156516446044036,
 'min_child_weight': 3}

In [9]:
best_params = study.best_params.copy()

model = xgb.XGBRegressor(objective="reg:squarederror",
                          n_jobs=-1,
                          grow_policy='lossguide',
                          tree_method="gpu_hist",
                          predictor="gpu_predictor",
                          booster='gbtree',
                          sampling_method='gradient_based',
                          eval_metrics=['rmse'],
                          random_state=RANDOM_SEED,
                          enable_categorical=False,
                          **best_params)
pipe = make_pipeline(StandardScaler(), model)
pipe.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('xgbregressor',
                 XGBRegressor(colsample_bytree=0.7000000000000001,
                              enable_categorical=False, eval_metrics=['rmse'],
                              gamma=0.1156516446044036, grow_policy='lossguide',
                              learning_rate=0.009707298411215315, max_depth=10,
                              min_child_weight=3, n_estimators=851, n_jobs=-1,
                              objective='reg:squarederror',
                              predictor='gpu_predictor', random_state=2,
                              sampling_method='gradient_based', subsample=1.0,
                              tree_method='gpu_hist'))])

In [10]:
train_pred = pipe.predict(X_train)
test_pred = pipe.predict(X_test)
print(f"train results - RMSE: {mean_squared_error(y_train, train_pred, squared=False)}, MAE: {mean_absolute_error(y_train, train_pred)}")
print(f"test results - RMSE: {mean_squared_error(y_test, test_pred, squared=False)}, MAE: {mean_absolute_error(y_test, test_pred)}")

train results - RMSE: 2.724727213443903, MAE: 1.3795214108377696
test results - RMSE: 4.063300824538348, MAE: 1.5649167010754348


In [11]:
mean_train = np.mean(y_train)
print(f"Baseline mean model - test RMSE: {np.sqrt(np.mean((y_test - mean_train)**2))}")
print(f"Baseline mean model - test MAE: {np.mean(np.abs(y_test - mean_train))}")

Baseline mean model - test RMSE: 4.685317144421096
Baseline mean model - test MAE: 1.9206922705000002
